In [411]:
import chainer
import chainer.functions as F
import chainer.links as L
import chainerrl
from chainerrl import explorers
import gym
import numpy as np

In [439]:
import pygame
import sys
from time import sleep
import random
import math
import numpy as np
from gym import spaces

# COLORS
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
purple = (255, 0, 255)

# SIZES
AGENT_SIZE = 50
WEAPON_SIZE = 20

class Entity:
    def __init__(self, name, xy, angle, speed, game_dims=(1000,800)):
        self.name = name
        self.x, self.y = xy
        self.speed = speed
        self.angle = math.radians(-angle)  # -1 to 1 
        self.dx = self.speed * math.cos(self.angle)
        self.dy = self.speed * math.sin(self.angle)
        self.dimx, self.dimy = game_dims
        
    def update(self, agent_xy):
        self.x += self.dx
        self.y += self.dy

        agent_x, agent_y = agent_xy

        has_hit_x = self.x >= agent_x - WEAPON_SIZE and self.x <= agent_x + AGENT_SIZE
        has_hit_y = self.y >= agent_y - WEAPON_SIZE and self.y <= agent_y + AGENT_SIZE
        
        exit_boundary = self.x > self.dimx-50 or self.x < 000 or self.y > self.dimy-50 or self.y < 0

        return has_hit_x and has_hit_y or exit_boundary
    
    def __repr__(self):
        return self.name + str((self.x,self.y))


class Agent:
    def __init__(self, xy=(400,100), game_dims=(1000,800), show=False):
        self.jumps = 0
        self.maxJumps = 2
        self.xpos, self.ypos = xy
        self.touchingObst = 0
        self.gravityPull = 0.5
        self.gravityCurrent = 0
        self.xCurrent = 0
        self.show = show
        self.dimx, self.dimy = game_dims
    def jump(self):
        if self.jumps < self.maxJumps:
            self.gravityCurrent = -10
            self.jumps = self.jumps + 1
    def left(self):
        if self.touchingObst == 0:
            self.xCurrent = -10
    def right(self):
        if self.touchingObst == 0:
            self.xCurrent = 10
    def update(self):
        # CONTROL GRAVITY
        self.gravityCurrent = self.gravityCurrent + self.gravityPull

        # RATE OF DECREASE OF LEFT/RIGHT MOVEMENTS
        if self.xCurrent > 0:
            self.xCurrent = self.xCurrent - 0.5
        if self.xCurrent < 0:
            self.xCurrent = self.xCurrent + 0.5

        # UPDATE XY COORDINATES
        self.ypos = self.ypos + self.gravityCurrent
        self.xpos = self.xpos + self.xCurrent

        # BOUNDARIES
        if self.xpos > self.dimx-50:
            self.xpos = self.dimx-50
        if self.xpos < 000:
            self.xpos = 000
        if self.ypos > self.dimy-50:
            self.ypos = self.dimy-50+1
            self.gravityCurrent = 0
            self.jumps = 0
            
    def display(self, gameDisplay):
        if self.show:
            pygame.draw.rect(gameDisplay, red, (self.xpos, self.ypos, AGENT_SIZE, AGENT_SIZE))
    def act(self, agent_action):
        # print(agent_action)
        if agent_action == 0:
            self.left()
        elif agent_action == 1:
            self.right()
        elif agent_action == 2:
            self.jump()
        self.update()
            

class Env:
    def __init__(self, 
                 game_dims=(1000, 800),
                 show=False):
        self.dimx, self.dimy = game_dims
        self.agent = Agent((400,100), show=show, game_dims=game_dims)
        self.set_default_rewards()
        self.observation_space = 5
        self.show = show
        pygame.init()
        self.play = True
        
        # GAME DIMENSIONS
        self.game_dims = game_dims
        self.generator_action_space = spaces.Box(np.array([0,0,0,0]), np.array([2,self.dimx,self.dimy,360]), dtype=np.float32)
        self.agent_action_space = spaces.Discrete(3)
        min_obs = np.array([0]*5 + [0,0,5,0]*5)
        max_obs = np.array([2,self.dimx,self.dimy,1,self.dimy*2] + [self.dimx,self.dimy,50,360]*5)
        self.observation_space = spaces.Box(min_obs, max_obs, dtype=np.float32)
        
        # DELAY FOR WEAPON ENTITIES
        self.delay = 0
#         self.entity_list = []
        
        self.entity_limit = 5
        self.entity_free_keys = [0,1,2,3,4]
        self.entity_dict = {}
    def get_free_key(self):
        if len(self.entity_free_keys) > 0:
            return self.entity_free_keys.pop(0)
        else:
            return None
    
    # CONTROL MOVEMENTS
    def execute(self):
        agent_action = None
        weapon_action = (0,0,0)
        
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    agent_action = 2
                if event.key == pygame.K_LEFT and self.agent.touchingObst == 0:
                    agent_action = 0
                if event.key == pygame.K_RIGHT and self.agent.touchingObst == 0:
                    agent_action = 1
            if event.type == pygame.QUIT:
                pygame.display.quit()
                agent_action = -1
                
        action = (agent_action, weapon_action)
        return action
            
    def test_agent(self):
        """FOR TESTING OF AGENT ACTIONS ONLY"""
        run = True
        while run:
            sleep(0.01)
            action = self.execute()
            if action == -1:
                break
            self.step(action)
            
    def create_entity(self, weapon_action):
        # print(weapon_action)
        wep_type, wep_x, wep_y, angle = weapon_action
        wep_xy = (wep_x, wep_y)
#         print(wep_xy)
        if self.delay != 0:
            self.delay -= 1
        wep_type = 1
        if wep_type == 1 and self.delay == 0:
            ent = Entity(str(wep_type), wep_xy, angle, 10)
            ent_key = self.get_free_key()
            if ent_key != None:
                self.entity_dict[ent_key] = ent
            self.delay = 20  # DELAY BEFORE THE NEXT ATTACK
    def update_entities(self):
        # UPDATE ENTITIES
        collided = []
        for key, ent in self.entity_dict.items():
            collide = ent.update((self.agent.xpos, self.agent.ypos))
            if not collide:
                if self.show:
#                     print(ent.x, ent.y)
                    pygame.draw.rect(self.gameDisplay, blue, (ent.x, ent.y, WEAPON_SIZE, WEAPON_SIZE))
            else:
                self.agent.agent_reward = -20
                self.generator_reward = 20
                collided.append(key)
        
        for ent_key in collided:
            self.entity_free_keys.append(ent_key)
            del self.entity_dict[ent_key]
    def display_game(self):
        if self.show:
#             print(self.game_dims)
            self.gameDisplay = pygame.display.set_mode(self.game_dims, 0, 32)
            self.gameDisplay.fill(white)
    def display_background(self):
        # DISPLAY BACKGROUND
        if self.show:
            pygame.font.init()
            myFont = pygame.font.SysFont('Futura PT Light', 60)
            textsurface = myFont.render('The Chosen One', False, black)
            self.gameDisplay.blit(textsurface, (200,200))
            pygame.display.update()
    def set_default_rewards(self):
        self.agent.agent_reward = 1
        self.generator_reward = -1

    def step(self, action):
        # SET DEFAULT REWARDS FOR AGENT AND GENERATOR
        self.set_default_rewards()
        
        # DISPLAY GAME
        self.display_game()
        
        agent_action, weapon_action = action
        
        # MOVE THE AGENT
        self.agent.act(agent_action)
        if self.show:
            self.agent.display(self.gameDisplay)
        
        # CREATE WEAPON ENTITY
        self.create_entity(weapon_action)
        
        # UPDATE ENTITIES
        self.update_entities()
        
        # DISPLAY BACKGROUND
        self.display_background()
        
        """RETURNS:
        reward - (agent_reward, generator_reward)
        state - getGameState()
        done - CURRENT: DEFAULT: False
        done - TODO: whether game is completed, e.g. HP <= 0
        """
        reward = (self.agent.agent_reward, self.generator_reward)
        state = self.getGameState()
        
        return (reward, state, False)
        
    def getGameState(self):
        a = self.agent
        agent_values = np.array([
            a.jumps,
            a.xpos//1000,
            a.ypos//1000,
            a.touchingObst,
            a.gravityCurrent,
            # TODO: height, width, dy, dx, direction, bounding box
        ])
        entity_values = np.array([])
        for i in range(5):
            if i in self.entity_dict:
                e = self.entity_dict[i]
                vals = [e.x//1000, e.y//1000, e.speed, e.angle]
            else:
                vals = [0,0,0,0]
            entity_values = np.append(entity_values, vals)

        values = np.append(agent_values, entity_values)
        return values
    
    def reset(self):
        """Resets the game. Returns (reward, state, done)."""
        self.__init__(game_dims=self.game_dims, show=self.show)
        return self.getGameState()

    def test_step(self):
        # Create Gun at random place and angles
        agent_action = random.randint(0,2)

        wep_type = 1  # gun
        wep_xy = (50, 700)  # coordinate appears at
        angle = 0

        generator_action = (wep_type, wep_xy, angle)
        action = (agent_action, generator_action)
        self.step(action)

In [440]:
env = Env(game_dims=(700, 500), show=False)

# SHOW ENVIRONMENT VALUES
print('Observation space:', env.observation_space)
print('Agent action space:', env.agent_action_space)
print('Generator action space:', env.generator_action_space)

state = env.reset()
print("State:", state)

Observation space: Box(25,)
Agent action space: Discrete(3)
Generator action space: Box(4,)
State: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


In [441]:
# Create Gun at random place and angles
agent_action = random.randint(0,2)

wep_type = 1  # gun
wep_xy = (50, 700)
wep_x, wep_y = wep_xy
angle = 20

generator_action = (wep_type, wep_x, wep_y, angle)
action = (agent_action, generator_action)
reward, state, done = env.step(action)
print("Agent Reward:", reward[0])
print("Generator Reward:", reward[1])
print("State:", state)
print("Done:", done)

Agent Reward: 1
Generator Reward: -1
State: [ 0.          0.          0.          0.          0.5         0.
  0.         10.         -0.34906585  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ]
Done: False


In [442]:
# FOR CONTINUOUS ACTION SPACE
obs_space = env.observation_space
obs_size = obs_space.low.size

In [443]:
"""AGENT HYPERPAREMETERS"""
agent_action_space = env.agent_action_space
agent_action_size = agent_action_space.n

# Q FUNCTION AND ADAM OPTIMIZER
agent_q_func = chainerrl.q_functions.FCStateQFunctionWithDiscreteAction(
    obs_size, agent_action_size,
    n_hidden_layers=5, n_hidden_channels=100)

# Use Adam to optimize q_func. eps=1e-2 is for stability.
agent_optimizer = chainer.optimizers.Adam(eps=1e-2)
agent_optimizer.setup(agent_q_func)

# Set the discount factor that discounts future rewards.
agent_gamma = 0.95

# Use epsilon-greedy for exploration
agent_explorer = chainerrl.explorers.ConstantEpsilonGreedy(
    epsilon=0.3, random_action_func=env.agent_action_space.sample)

# DQN uses Experience Replay.
# Specify a replay buffer and its capacity.
agent_replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10 ** 6)

# Since observations from CartPole-v0 is numpy.float64 while
# Chainer only accepts numpy.float32 by default, specify
# a converter as a feature extractor function phi.
agent_phi = lambda x: x.astype(np.float32, copy=False)

# CHOSEN ONE AGENT
chosen_one = chainerrl.agents.DoubleDQN(
    agent_q_func, agent_optimizer, agent_replay_buffer, agent_gamma, agent_explorer,
    replay_start_size=500, update_interval=1,
    target_update_interval=100, phi=agent_phi)

In [444]:
"""GENERATOR HYPERPARAMETERS"""
generator_action_space = env.generator_action_space
generator_action_size = generator_action_space.low.size

# Q FUNCTION FOR CONTINUOUS VARIABLES
generator_q_func = chainerrl.q_functions.FCQuadraticStateQFunction(
    obs_size, generator_action_size,
    n_hidden_layers=5,
    n_hidden_channels=100,
    action_space=generator_action_space,
)

# Use the Ornstein-Uhlenbeck process for exploration
generator_ou_sigma = (generator_action_space.high - generator_action_space.low) * 0.2
generator_explorer = explorers.AdditiveOU(sigma=generator_ou_sigma)

# Use Adam to optimize q_func. eps=1e-2 is for stability.
generator_optimizer = chainer.optimizers.Adam(eps=1e-2)
generator_optimizer.setup(generator_q_func)

# Set the discount factor that discounts future rewards.
generator_gamma = 0.95

# DQN uses Experience Replay.
# Specify a replay buffer and its capacity.
generator_replay_buffer = chainerrl.replay_buffer.ReplayBuffer(capacity=10 ** 6)

# Since observations from CartPole-v0 is numpy.float64 while
# Chainer only accepts numpy.float32 by default, specify
# a converter as a feature extractor function phi.
generator_phi = lambda x: x.astype(np.float32, copy=False)

# GENERATOR AGENT
generator = chainerrl.agents.DoubleDQN(
    generator_q_func, generator_optimizer, generator_replay_buffer, generator_gamma, generator_explorer,
    replay_start_size=500, update_interval=1,
    target_update_interval=100, phi=generator_phi)

In [445]:
agent_action = chosen_one.act_and_train(obs, agent_reward)
generator_action = generator.act_and_train(obs, generator_reward)
action = (agent_action, generator_action)

(reward, obs, done) = env.step(action)
agent_reward, generator_reward = reward

In [ ]:
# MAIN TRAINING LOOP
n_episodes = 1000
max_episode_len = 500

R_agent_history = []
R_generator_history = []
for i in range(1, n_episodes + 1):
    obs = env.reset()
    agent_reward = 0
    generator_reward = 0
    done = False
    R_agent = 0  # return (sum of rewards)
    R_generator = 0
    t = 0  # time step
    while not done and t < max_episode_len:
        # Uncomment to watch the behaviour
        # env.render()
#         print(obs)
        agent_action = chosen_one.act_and_train(obs, agent_reward)
        generator_action = generator.act_and_train(obs, generator_reward)
        action = (agent_action, generator_action)
        
        (reward, obs, done) = env.step(action)
        agent_reward, generator_reward = reward

        R_agent += agent_reward
        R_generator += generator_reward
        R_agent_history.append(R_agent)
        R_generator_history.append(R_generator)
        t += 1
    if i % 1 == 0:
        print('ep:', i,
              'R_agent:', R_agent,
#               'R_generator:', R_generator,
              'agent stat:', chosen_one.get_statistics())
#               'generator statistics:', generator.get_statistics())
    chosen_one.stop_episode_and_train(obs, agent_reward, done)
    generator.stop_episode_and_train(obs, generator_reward, done)
print('Finished.')

ep: 1 R_agent: 17 agent stat: [('average_q', 8.933350225869424), ('average_loss', 1.0534251732671915), ('n_updates', 2219)]
ep: 2 R_agent: -4 agent stat: [('average_q', 10.506435539273403), ('average_loss', 0.932203403650184), ('n_updates', 2719)]
ep: 3 R_agent: 17 agent stat: [('average_q', 12.344744379204764), ('average_loss', 1.0980929290231225), ('n_updates', 3219)]
ep: 4 R_agent: 17 agent stat: [('average_q', 13.576695332067278), ('average_loss', 1.013638625438399), ('n_updates', 3719)]
ep: 5 R_agent: -4 agent stat: [('average_q', 14.52009680853569), ('average_loss', 0.9688254540884538), ('n_updates', 4219)]
ep: 6 R_agent: -25 agent stat: [('average_q', 15.15822366882791), ('average_loss', 1.0232812214119211), ('n_updates', 4719)]
ep: 7 R_agent: -4 agent stat: [('average_q', 15.958815863795643), ('average_loss', 1.0872832611125127), ('n_updates', 5219)]
ep: 8 R_agent: 38 agent stat: [('average_q', 16.49151832260835), ('average_loss', 0.8900419801770801), ('n_updates', 5719)]
ep: 9